아래 코드는 버전 오류가 뜨면 실행해주세요

In [1]:
#pip install keras==2.12.0
#conda install scikit-learn

In [1]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')
tf.config.set_visible_devices([], 'GPU')

여기는 실행하지 않아도 됩니다. 제가 가진 파일과 동일한지 확인용입니다.
아래의 shape, columns가 동일한지 확인하여 진행하시면 됩니다.

In [2]:
df = pd.read_csv("modified_0419.csv")

In [3]:
df.shape

(1131682, 92)

In [4]:
df.head()

,Unnamed: 0,loan_amnt,term,int_rate,installment,sub_grade,emp_length,annual_inc,loan_status,dti,...,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,0,6000.0,36,7.97,187.94,4,2,45000.0,0,8.67,...,False,False,False,False,False,False,False,False,False,True
1,1,23200.0,60,24.99,680.82,23,10,110000.0,1,34.70,...,False,False,False,False,False,False,False,False,False,True
2,2,16000.0,36,7.07,494.55,1,10,65000.0,1,17.74,...,False,False,False,False,False,False,False,True,False,False
3,3,4500.0,36,10.42,146.10,7,5,50000.0,0,6.99,...,False,False,False,False,False,False,False,True,False,False
4,4,20000.0,36,9.99,645.25,7,10,60000.0,0,28.94,...,False,False,False,False,False,False,False,False,True,False


In [5]:
'addr_state' in df.columns

False

In [6]:
'loan_amnt_intallment_rate' in df.columns

False

In [7]:
df.columns

Index(['Unnamed: 0', 'loan_amnt', 'term', 'int_rate', 'installment',
       'sub_grade', 'emp_length', 'annual_inc', 'loan_status', 'dti',
       'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'collections_12_mths_ex_med', 'application_type', 'annual_inc_joint',
       'dti_joint', 'acc_now_delinq', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'total_cu_tl', 'bc_open_to_buy',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       '

여기부터 데이터 처리 시작, 실행해주세요

In [7]:
import pandas as pd
import numpy as np
from typing import Union, List
'''
도은수정 - 0420 버전
- 결측 처리 완료
- 로그변환 추가
- 다중공선성 : fico_avg 변수 추가
'''
class Preprocessor0420:
    def __init__(self):
        self.file_path = ""
        self.folder_path = ""
        self.df = pd.DataFrame()

    def __init__(self, data_file_path:str="", folder_path:str=""):
        self.file_path = data_file_path
        self.folder_path = folder_path
        self.df = pd.DataFrame()

    def load_origin_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)

    def load_modified_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)
        if 'Unnamed: 0' in self.df.columns:
            self.df.drop(columns=['Unnamed: 0'], inplace=True)

    def drop_columns(self, drop_columns_file_path:str)->None:
        return None

    def __preprocess_target_variable(self, target_variable:str="loan_status")->None:
        return None

    def __fill_na_with_value(self, columns:List[str], filling_value:Union[str, int])->None:
        '''
        df: dataframe to fill NA
        column_name : column name to change NA values
        filling_value : value type or just value to fill column's NA
        '''
        # for column_name in columns:
        #     if filling_value=="mode":
        #         mode_value = self.df[column_name].mode()[0]
        #     elif filling_value=="median":
        #         mode_value = self.df[column_name].median()
        #     else:
        #         mode_value = filling_value
        #     self.df[column_name].fillna(mode_value, inplace=True)
        return None

    def __preprocessing_na(self)->None:
        ## 결측 처리
        # 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
        # A1. 최빈값 대체
        # A2. 중앙값 대체
        # B. 2015년 대체
        # C. 2012년 대체
        # D. 결측 0 대체
        return None

    def __Multicollinearity(self)->None:
        # self.df.drop(columns=['fico_range_low'], inplace=True)
        self.df['fico_avg'] = (self.df['fico_range_low'] + self.df['fico_range_high'])/2
        self.df.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)

    def __convert_log(self, term):
        return np.log(term + 1)

    def __log_transform(self):
        variables = [
          "all_util", "annual_inc", "annual_inc_joint", "bc_open_to_buy", # "avg_cur_bal",
          "delinq_amnt", "dti", "max_bal_bc", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op",
          "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc", "mths_since_rcnt_il",
          "mths_since_recent_bc", "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl",
          "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl", "num_rev_accts",
          "num_rev_tl_bal_gt_0", "num_sats", "open_acc", "open_acc_6m", "open_act_il",
          "open_il_12m", "open_il_24m", "open_rv_12m", "open_rv_24m", "pub_rec_bankruptcies",
          "revol_bal", "revol_bal_joint", "tax_liens", "tot_cur_bal", "tot_hi_cred_lim",
          "total_acc", "total_bal_ex_mort", "total_bal_il", "total_bc_limit", "total_cu_tl",
          "total_il_high_credit_limit", "total_rev_hi_lim"
        ]

        # 각 변수에 대해 로그 변환 수행 및 기존 변수 삭제
        for var in variables:
            # 로그 변환 후 변수 이름에 '_log'를 추가하여 새로운 변수 생성
            new_var = var + "_log"
            # 해당 변수가 0보다 큰 경우에만 로그 변환 수행하여 음수 무한대를 방지
            # 로그 변환 후에는 기존 값이 0인 경우 음수 무한대로 처리되므로 이에 대한 처리도 필요
            # 여기서 로그는 자연로그 (밑이 e인 로그)
            self.df[new_var] = np.log(self.df[var] + 1)  # 0이 아닌 값이어야 하므로 +1 추가
            # 기존 변수 삭제
            self.df.drop(columns=[var], inplace=True)
        return self.df

    def __convert_object_to_numeric(self, column_name:str)->pd.DataFrame:
        return None

    def __convert_object_to_one_hot(self, column_name:str)->None:
        return None

    def __preprocessing_objects(self)->None:
        return None

    def preprocess(self)->None:
        # loan_status 제외 모든 column이 결측치(na)인 행 제거 (1개 행 제거됨)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)
        self.__preprocess_target_variable()
        # 결측치 제거
        self.__preprocessing_na()
        ## object 처리하기
        self.__preprocessing_objects()
        ## 다중공선성 제거 - 0419 추가
        self.__Multicollinearity()
        ## 로그변환 - 0419 추가
        self.__log_transform()
        # index 재설정
        self.df.reset_index(drop=True, inplace=True)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)

    def get_df(self)->pd.DataFrame:
        return self.df

In [8]:
p = Preprocessor0420()
# lending_club_2020_train.csv 파일이 있는 절대 경로 혹은 상대 경로를 명시해주세요
p.load_modified_file(file_path="modified_0419.csv")
# preprocess를 돌리기
p.preprocess()

shape와 columns 확인

In [9]:
df = p.get_df()
df.shape

(1131682, 90)

In [23]:
df.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade',
       'emp_length', 'loan_status', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec',
       'revol_util', 'collections_12_mths_ex_med', 'application_type',
       'dti_joint', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'mths_since_recent_inq', 'num_tl_120dpd_2m', 'num_tl_30dpd',
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'is_after_2015', 'is_after_2012', 'MORTGAGE', 'OTHERS', 'OWN', 'RENT',
       'car', 'credit_card', 'debt_consolidation', 'educational',
       'home_improvement', 'house', 'major_purchase', 'medical', 'moving',
       'other', 'renewable_energy', 'small_business', 'vacation', 'wedding',
       'Not Verified', 'Source Verified', 'Verified', 'fico_avg',
       'all_util_log', 'annual_inc_log', 'annual_inc_joint_log',
       'bc_open_to_buy_log', 'delinq_amnt_log', 'dti_log', 'max_bal_bc_log',
       'mo_sin_old_il_acct_log', 'mo_sin_old_rev_tl_op_log',
       'mo_si

In [24]:
df.head()

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,loan_status,delinq_2yrs,inq_last_6mths,pub_rec,...,tax_liens_log,tot_cur_bal_log,tot_hi_cred_lim_log,total_acc_log,total_bal_ex_mort_log,total_bal_il_log,total_bc_limit_log,total_cu_tl_log,total_il_high_credit_limit_log,total_rev_hi_lim_log
0,6000.0,36,7.97,187.94,4,2,0,1.0,0.0,0.0,...,0.0,10.820878,11.272280,3.610918,10.820878,10.757158,9.615872,0.000000,10.942208,10.003378
1,23200.0,60,24.99,680.82,23,10,1,1.0,1.0,0.0,...,0.0,13.315176,13.386134,4.110874,12.592905,12.533767,9.918425,1.386294,12.582679,10.317450
2,16000.0,36,7.07,494.55,1,10,1,0.0,0.0,0.0,...,0.0,11.522113,12.186584,3.367296,10.390594,10.263641,10.987003,0.000000,10.806490,11.157692
3,4500.0,36,10.42,146.10,7,5,0,0.0,0.0,2.0,...,0.0,12.178635,12.263577,2.302585,9.166806,7.919356,9.137877,0.000000,9.390242,9.532496
4,20000.0,36,9.99,645.25,7,10,0,0.0,0.0,0.0,...,0.0,11.831590,12.137612,3.713572,11.101704,0.000000,9.305741,0.000000,11.199461,10.229224


In [10]:
'int_rate' in df.columns

True

데이터 처리 완료!

In [11]:
x_train = df.drop(columns=['loan_status'])
y_train = df['loan_status']